In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import os

# LLM Layer (RAG Chatbot)

This notebook sets up the final **LLM layer** for our Retrieval-Augmented Generation (RAG) chatbot. 

✅ It connects to a local Chroma DB containing vector embeddings of the documents.  
✅ Uses a lightweight open-source model (**Flan-T5**) via HuggingFace Pipeline for natural language generation.  
✅ Combines both for a **conversational chatbot** that:
- Retrieves relevant document chunks from Chroma DB.
- Generates contextual answers with LLM.
- Maintains conversational memory for a natural chat experience.

This is the last step of our RAG pipeline, enabling us to answer user questions interactively and conversationally.


In [ ]:
# Initialize Chroma retriever
persist_directory = "chroma_db"
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

chroma_db = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_model
)
retriever = chroma_db.as_retriever()

In [ ]:
# Groq API function
import requests

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

def get_groq_completion(prompt: str) -> str:
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {GROQ_API_KEY}"
    }
    data = {
        "model": "deepseek-r1-distill-llama-70b",
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are a helpful assistant. Provide clear, concise, and professional answers. The other thing to consider is potential buyers might interact with you so you should subtly sell the product without being too pushy. "
                )
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    }
    response = requests.post(
        "https://api.groq.com/openai/v1/chat/completions",
        headers=headers,
        json=data
    )
    response.raise_for_status()
    result = response.json()["choices"][0]["message"]["content"]


    if "<think>" in result and "</think>" in result:
        result = result.split("</think>")[-1].strip()

    return result.strip()

In [ ]:
# Final context-aware QA function
def answer_with_context(question: str, top_k: int = 10):
    top_chunks: list[Document] = retriever.get_relevant_documents(question, top_k=top_k)
    combined_context = "\n\n".join(chunk.page_content for chunk in top_chunks)

    prompt = (
        f"Context:\n{combined_context}\n\n"
        f"Question: {question}\n\n"
        f"Answer concisely and professionally."
    )
    answer = get_groq_completion(prompt)
    return answer


In [ ]:
question = "How do I integrate the Rockfish API in my existing stack? give me clear and concise code examples. Imagine that I have the API key"
answer = answer_with_context(question)
print("\nAnswer:\n", answer)

In [ ]:
question = "What types of data can it create? Can it create data for a specific industry?"
answer = answer_with_context(question)
print("\nAnswer:\n", answer)

In [ ]:
question = "what is the research behind the product? What is the technology behind the product? who are the main researchers behind the product?"
answer = answer_with_context(question)
print("\nAnswer:\n", answer)

In [ ]:
question = "can you cite some technicalities of the product from a research standpoint? What are the main technical features of the product?"
answer = answer_with_context(question)
print("\nAnswer:\n", answer)

In [ ]:
question = "what do you customers say about the product? Give concrete examples and cite the sources. How do I know whether the product creates useful data?"
answer = answer_with_context(question)
print("\nAnswer:\n", answer)

In [ ]:
question = "I am on the fence about buying the product. Can you convince me to buy it? Why rockfish?"
answer = answer_with_context(question)
print("\nAnswer:\n", answer)

In [ ]:
question = "tell me about your latest presence in the news. What are the latest developments in the company? Give concrete events and cite the sources."
answer = answer_with_context(question)
print("\nAnswer:\n", answer)